In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

### Pre-Activation ResNet은 Basic Block과 BottleNeck Blcok을 약간만 수정하면 구현 가능하다.

<img src='./pre-activation.png' width="550" height="350">

In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1, bias=False)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)

        self.idenity = nn.Sequential()

        if stride != 1 or in_channel != out_channel * 4:
            self.idenity = nn.Sequential(
                nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        x_res = F.relu(self.bn1(x_res))
        x_res = self.conv1(x)

        x_res = F.relu(self.bn2(x_res))
        x_res = self.conv2(x_res)

        x_out = x_res + self.idenity(x)

        return x_out

In [4]:
class BottleNeck(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(BottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv3 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel * 4, kernel_size=1, stride=1, padding=0, bias=False)

        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.bn3 = nn.BatchNorm2d(out_channel * 4)

        self.identity = nn.Sequential()

        if stride != 1 or in_channel != out_channel * 4:
            self.identity = nn.Sequential(
                nn.Conv2d(in_channels=in_channel, out_channels=out_channel * 4, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(out_channel * 4)
            )
    
    def forward(self, x):
        x_res = F.relu(self.bn1(x_res))
        x_res = self.conv1(x)

        x_res = F.relu(self.bn2(x_res))
        x_res = self.conv2(x_res)

        x_res = F.relu(self.bn3(x_res))
        x_res = self.conv3(x_res)

        x_out = x_res + self.identity(x)
        
        return x_out

In [5]:
class PreAct_ResNet(nn.Module):
    def __init__(self, block, num_block):
        super(PreAct_ResNet, self).__init__()

        self.in_channel = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2 = self._make_layers(block, 64, num_block[0], 1)
        self.conv3 = self._make_layers(block, 128, num_block[1], 2)
        self.conv4 = self._make_layers(block, 256, num_block[2], 2)
        self.conv5 = self._make_layers(block, 512, num_block[3], 2)

        if block.__name__ == 'BasicBlock':
            self.classifier = nn.Sequential(
                nn.AvgPool2d(kernel_size=(7,7)),
                nn.Flatten(),
                nn.Linear(in_features=512, out_features=1000)
            )
        else:
            self.classifier = nn.Sequential(
                nn.AvgPool2d(kernel_size=(7,7)),
                nn.Flatten(),
                nn.Linear(in_features=2048, out_features=1000)
            )

    def _make_layers(self, block, out_channel, num_block, stride):
        strides = [stride] + [1] * (num_block - 1)

        layers = []
        for stride in strides:
            layers.append(block(self.in_channel, out_channel, stride))

            if block.__name__ == 'BasicBlock':
                self.in_channel = out_channel
            else:
                self.in_channel = out_channel * 4
                
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.classifier(x)

        return x

In [6]:
model = PreAct_ResNet(BottleNeck, [3, 8, 36, 3])

In [7]:
model

PreAct_ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): BottleNeck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (identity): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=

In [45]:
summary(model, (32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 1000]                --
├─Sequential: 1-1                        [32, 64, 56, 56]          --
│    └─Conv2d: 2-1                       [32, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                  [32, 64, 112, 112]        128
│    └─ReLU: 2-3                         [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                    [32, 64, 56, 56]          --
├─Sequential: 1-2                        [32, 256, 56, 56]         --
│    └─BottleNeck: 2-5                   [32, 256, 56, 56]         --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          4,096
│    │    └─BatchNorm2d: 3-2             [32, 64, 56, 56]          128
│    │    └─Conv2d: 3-3                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-4             [32, 64, 56, 56]          128
│    │    └─Conv2d: 3-5                  [32, 256, 56, 56]         16,38

In [46]:
from torchvision.models import resnet152

In [48]:
model = resnet152()

In [49]:
summary(model, (32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 1000]                --
├─Conv2d: 1-1                            [32, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        128
├─ReLU: 1-3                              [32, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [32, 64, 56, 56]          --
├─Sequential: 1-5                        [32, 256, 56, 56]         --
│    └─Bottleneck: 2-1                   [32, 256, 56, 56]         --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          4,096
│    │    └─BatchNorm2d: 3-2             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-3                    [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-5             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-6                    [32, 64, 56, 56]          --
│ 